In [ ]:
import pandas as pd
from pyvis.network import Network

# Load data from Excel
file_path = '14 36 ENG Expanded_RO_with_Supporting_Cryptocurrencies.xlsx'
df = pd.read_excel(file_path)

# Initialize the network
net = Network(notebook=True, height="750px", width="100%")

# To store added nodes and connections
added_nodes = set()
learning_outcomes_topics = {}
specialty_outcomes = {}
topic_outcomes = {}
crypto_properties_outcomes = {}
crypto_types_properties = {}

# Add nodes and edges between Specialties, Learning Outcomes, Themes, and Cryptocurrencies
specialties = df['Specialty'].unique()

# Add nodes for Specialties
for specialty in specialties:
    specialty = str(specialty)
    net.add_node(specialty, label=specialty, shape='circle', color='pink', title=f'Specialty: {specialty}\nRelated Learning Outcomes:', group='Specialty')
    added_nodes.add(specialty)
    specialty_outcomes[specialty] = set()

# Add nodes and edges for Learning Outcomes, Themes, Crypto Properties, and Crypto Types
for index, row in df.iterrows():
    specialty = str(row['Specialty'])
    outcome = str(row['Learning_Outcome'])
    topic = str(row['Theme'])
    crypto_property = str(row['Crypto_Properties'])
    crypto_type = str(row['Supporting_Cryptocurrencies'])
    
    # Add Learning Outcomes nodes and update connections
    if outcome not in added_nodes:
        net.add_node(outcome, label=outcome, shape='box', color='#33FF57', title=f"Learning Outcome: {outcome}\nRelated Themes:\n", group='Learning Outcome')
        added_nodes.add(outcome)
        learning_outcomes_topics[outcome] = set()
    
    learning_outcomes_topics[outcome].add(topic)
    specialty_outcomes[specialty].add(outcome)
    net.add_edge(specialty, outcome)
    
    # Add Theme nodes and update connections
    if topic not in added_nodes:
        net.add_node(topic, label=topic, shape='ellipse', color='lightgreen', title=f'Theme: {topic}\nRelated Learning Outcomes:', group='Theme')
        added_nodes.add(topic)
        topic_outcomes[topic] = set()
    
    topic_outcomes[topic].add(outcome)
    net.add_edge(outcome, topic)
    
    # Add Crypto Properties nodes and connections
    if crypto_property not in added_nodes:
        net.add_node(crypto_property, label=crypto_property, shape='dot', size=10, color='pink', title=f'Crypto Property: {crypto_property}\nRelated Learning Outcomes:\n', group='Crypto Property')
        added_nodes.add(crypto_property)
        crypto_properties_outcomes[crypto_property] = set()
    
    crypto_properties_outcomes[crypto_property].add(outcome)
    net.add_edge(outcome, crypto_property)
    
    # Add Crypto Types nodes and connections
    if crypto_type not in added_nodes:
        net.add_node(crypto_type, label=crypto_type, shape='dot', size=8, color='blue', title=f'Crypto Type: {crypto_type}\nRelated Property: {crypto_property}', group='Crypto Type')
        added_nodes.add(crypto_type)
        crypto_types_properties[crypto_type] = set()
    
    crypto_types_properties[crypto_type].add(crypto_property)
    net.add_edge(crypto_property, crypto_type)

# Update tooltips (titles) for Learning Outcomes
for outcome, topics in learning_outcomes_topics.items():
    net.get_node(outcome)['title'] += "\n".join(topics)

# Update tooltips (titles) for Specialties
for specialty, outcomes in specialty_outcomes.items():
    net.get_node(specialty)['title'] += "\n".join(outcomes)

# Update tooltips (titles) for Themes
for topic, outcomes in topic_outcomes.items():
    net.get_node(topic)['title'] += "\n".join(outcomes)

# Update tooltips (titles) for Crypto Properties
for crypto_property, outcomes in crypto_properties_outcomes.items():
    related_crypto_types = [crypto_type for crypto_type, properties in crypto_types_properties.items() if crypto_property in properties]
    net.get_node(crypto_property)['title'] += "\nRelated Crypto Types:\n" + "\n".join(related_crypto_types)

# Update tooltips (titles) for Crypto Types
for crypto_type, properties in crypto_types_properties.items():
    net.get_node(crypto_type)['title'] += "\n".join(properties)

# Save the network to an HTML file
net.show('specialties_learning_outcomes_topics_crypto.html')

# Adding buttons and click events to the HTML
html_file = 'specialties_learning_outcomes_topics_crypto.html'
with open(html_file, 'r') as file:
    html_content = file.read()

# JavaScript for interactive filtering by clicking on nodes
js_code = """
<script>
var nodesDataset = network.body.data.nodes;

function filterByNode(nodeId) {
    var connectedNodes = network.getConnectedNodes(nodeId);
    nodesDataset.update(nodesDataset.get().map(function(node) {
        node.hidden = !connectedNodes.includes(node.id) && node.id !== nodeId;
        return node;
    }));
}

network.on("click", function (params) {
    if (params.nodes.length > 0) {
        var clickedNodeId = params.nodes[0];
        filterByNode(clickedNodeId);
    }
});
</script>

<div style="margin-bottom: 20px;">
    <button onclick="nodesDataset.update(nodesDataset.get().map(function(node) { node.hidden = false; return node; }))">Show All</button>
</div>
"""

# Insert the JavaScript and buttons before the </body> tag
html_content = html_content.replace('</body>', js_code + '</body>')

# Save the modified HTML
with open(html_file, 'w') as file:
    file.write(html_content)
